In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 865.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.2 MB/s eta 0:00:0000:010:01m


In [6]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.5 MB/s eta 0:00:00a 0:00:010m


In [1]:
import pandas as pd
fake_train = pd.read_csv("fake_train_inc_partial_backtranslated_aug_indexed_useful_no_error_to_aug.csv").iloc[:, -3:]
fake_train

/tmp/ipykernel_27138/2902909327.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,text,misinformation,index
0,"Esse Psiquiatra analisando uma ""mente esquerdo...",1,31
1,QUEM SÃO OS CRITICOS DO BOLSONARO? V...,1,154
2,"Começou a perseguição aos cristãos, católicos ...",1,204
3,"Lindemberg Faria, e Benedita da Silva, reúnem ...",0,228
4,*🔫Você sabia que DESARMAMENTO da população não...,1,298
...,...,...,...
558,Bloco de paródia evangélica “Segura o Culto” e...,1,16758
559,PF procurou Lula no sítio antes de ir para sua...,1,16764
560,"Branco e nulo, Alckmin comemora resultados do ...",1,16765
561,MST invade faixa olímpica e declara latifúndio...,1,16771


In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to("cuda")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [3]:
def backtranslate_text_en_to_pt(text: str):
  tokenizer.src_lang = "pt_XX"
  encoded_pt = tokenizer(text, return_tensors="pt").to("cuda")
  generated_tokens = model.generate(
      **encoded_pt,
      forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
  )
  backtranslate = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  tokenizer.src_lang = "en_XX"
  encoded_en = tokenizer(backtranslate, return_tensors="pt").to("cuda")
  generated_tokens = model.generate(
      **encoded_en,
      forced_bos_token_id=tokenizer.lang_code_to_id["pt_XX"]
  )
  del encoded_en['input_ids']
  del encoded_en['attention_mask']
  del encoded_pt['input_ids']
  del encoded_pt['attention_mask']
  del encoded_en
  del encoded_pt
  backtranslated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  del generated_tokens
  return backtranslated_text

In [4]:
pairs = [(x, x+100) if x + 100 < fake_train.shape[0] else (x, fake_train.shape[0])  for x in range(0,fake_train.shape[0], 100)]
pairs

[(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 563)]

In [6]:
for pair in pairs:
    backtranslated_text = pd.DataFrame({"text": [], "misinformation": []})
    for index, row in fake_train[pair[0]:pair[1]].iterrows():
        if len(row["text"].split()) <= 200:
            backtranslated = [
              backtranslate_text_en_to_pt(row["text"])[0],
              row["misinformation"]
            ]
            backtranslated_text.loc[backtranslated_text.shape[0]] = backtranslated
    backtranslated_text.to_csv(f"./fake_train_inc_{pair[0]}_{pair[1]}_backtranslated_aug_partial.csv")

In [8]:
fake_partial = pd.read_csv("./fake_train_inc_0_100_backtranslated_aug_partial.csv")
for pair in pairs[1:]:
    fake_partial = pd.concat([fake_partial,pd.read_csv(f"./fake_train_inc_{pair[0]}_{pair[1]}_backtranslated_aug_partial.csv")])

fake_partial

,Unnamed: 0,text,misinformation
0,0,Este psiquiatra que está a analisar a esclavit...,1
1,1,O que são todos os críticos do Bolsonaro? Por ...,1
2,2,"Ele começou a perseguir cristãos, católicos e ...",1
3,3,"Lindemberg Faria, e Benedicta da Silva, reúnem...",0
4,4,Sabiam que a desmobilização da população não t...,1
...,...,...,...
58,58,"O bloco de desfile evangélico chamado """" Secur...",1
59,59,O PF queria que Lula estivesse no local antes ...,1
60,60,"Negro e branco, Alckmin comemora os resultados...",1
61,61,O MST invade a faixa Olímpica e declara uma gr...,1


In [11]:
fake_partial["index"] = fake_train["index"].values
fake_partial

,Unnamed: 0,text,misinformation,index
0,0,Este psiquiatra que está a analisar a esclavit...,1,31
1,1,O que são todos os críticos do Bolsonaro? Por ...,1,154
2,2,"Ele começou a perseguir cristãos, católicos e ...",1,204
3,3,"Lindemberg Faria, e Benedicta da Silva, reúnem...",0,228
4,4,Sabiam que a desmobilização da população não t...,1,298
...,...,...,...,...
58,58,"O bloco de desfile evangélico chamado """" Secur...",1,16758
59,59,O PF queria que Lula estivesse no local antes ...,1,16764
60,60,"Negro e branco, Alckmin comemora os resultados...",1,16765
61,61,O MST invade a faixa Olímpica e declara uma gr...,1,16771


In [12]:
import pandas as pd
fake_train_full_b_aug = pd.read_csv("./fake_train_inc_full_backtranslated_aug_indexed_useful.csv").iloc[:, -3:]
fake_train_full_b_aug = pd.concat([fake_train_full_b_aug, fake_partial.iloc[:, -3:]])

In [14]:
fake_train_full_b_aug = fake_train_full_b_aug.sort_values(by=['index'])
fake_train_full_b_aug.to_csv("fake_train_inc_full_backtranslated_aug_indexed_balanced_v1.csv")
fake_train_full_b_aug

,text,misinformation,index
0,Gostaria de pedir a todos os grupos neste LINK...,0,2
1,"O Brasil que quero, um Brasil com mais FRANTIs...",0,3
2,Porque o GLOBBY não mostra... BOLSONARO O MEU ...,0,4
3,É assim. O Chef do restaurante aparece no víde...,1,5
4,"Duas das maiores empresas de venda no mundo, a...",0,8
...,...,...,...
58,"O bloco de desfile evangélico chamado """" Secur...",1,16758
59,O PF queria que Lula estivesse no local antes ...,1,16764
60,"Negro e branco, Alckmin comemora os resultados...",1,16765
61,O MST invade a faixa Olímpica e declara uma gr...,1,16771


In [ ]:
#fake_partial.to_csv("fake_train_inc_full_paraphrased_aug_indexed.csv")